## Setup

For this notebook, we'll use the commitment transaction created in `chapter 3 - in-flight htlc commitment transaction` which contains the offered HTLC output that we'll spend with the HTLC-timeout transaction.

In [ ]:
%run "../chapter 3 - in-flight htlc commitment transaction/in-flight htlc commitment transaction.ipynb"

# Exercise: Creating the Bob In-Flight HTLC Commitment Transaction

Now as exercise let's create the Bob In-Flight Commitment Transaction.

## The Unsigned Transaction

### The Input

The input is the same.

### The Outputs

Bob transaction will have 4 outputs also:
* local anchor output
* remote anchor output
* accepted htlc output
* to remote output

#### The Accepted HTLC Output

    +------+---------------+
    | OP_1 |       Q       |
    +------+---------------+
                   ^
                   |   +-------------------+
                    ---| P(revocation) + T |
                       +-------------------+
                                         ^
                                         |
                                   +-----------+        
                                   | T = t * G |
                                   +-----------+        
                                         ^
                                         |
     +---+   +-------------------------------------------------------+
     | t | = | TaggedHash ("Taptweak", P(revocation) || script_root) |
     +---+   +-------------------------------------------------------+
                                                             ^
                                                             |
                                                          +-----+
                          ------------------------------> | hAB |<------------------
                         |                                +-----+                   |
                         |                                                          |
                         |                                                          |
                      +----+                                                     +----+
                      | hA |                                                     | hB |
                      +----+                                                     +----+
                         ^                                                          ^
                         |                                                          |
      +----------------------------------------------------+                        |
      | P(remote_htlc) OP_CHECKSIG OP_1 OP_CSV OP_DROP     |                        |
      |                                                    |                        |
      | cltv_expiry OP_CHECKLOCKTIMEVERIFY OP_DROP         |                        |
      +----------------------------------------------------+                        |
      +-------------------------------------------------------------------------------+ 
      | OP_SIZE 32 OP_EQUALVERIFY OP_HASH160 <RIPEMD160(payment_hash)> OP_EQUALVERIFY |
      |                                                                               |
      | P(local_htlc) OP_CHECKSIGVERIFY P(remote_htlc) OP_CHECKSIG                    |
      +-------------------------------------------------------------------------------+


In [ ]:
# Create Alice Local Public Key which in this case is the P(remote) for Bob
# TODO: Use the method 'get_pubkey' from the basepoint opject (alice payment) and the alice per commitment point to derive the alice payment pubkey

# TODO: Use the derivate_key class and bob per commitment point to derive the bob delayed pubkey

# TODO: Use the derivate_revocation_key and alice per commitment point to derive Alice Revocation Public Key

# Outputs for Bob In-Flight Commitment Transaction
# 0x04 outputs
output_count = bytes.fromhex("04")

# ANCHOR OUTPUTS
# Anchor output amount (we use the same amount as before)

# OP_16 OP_CSV
# TODO: Use CScript to create the anchor script

# Method: ser_string(data) is a function which adds compactsize to input data.
hash_input = TAPSCRIPT_VER + ser_string(script)

# Alice Anchor Output script_root
# TODO: Use the tagged_hash function to compute the script root

# Alice Anchor Output Tagged Hash (remote anchor for Bob)
# TODO: Use the tagged_hash function to compute the taptweak

# TODO: Use the tweak_add to tweak the alice payment pubkey

# TODO: Create scriptPubKey (alice_anchor_spk) P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)

# Bob Anchor Output Tagged Hash (local anchor for Bob)
# TODO: Use the tagged_hash function to compute the taptweak

# TODO: Use the tweak_add to tweak the bob delayed pubkey

# TODO: Use scriptPubKey (bob_anchor_spk) P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)

# Sort by scriptPubKey (lexicographic order)
anchors = [
    (alice_anchor_spk, "alice_anchor"),
    (bob_anchor_spk,   "bob_anchor"),
]

anchors_sorted = sorted(anchors, key=lambda x: x[0])

# ACCEPTED HTLC OUTPUT
# HTLC output amount (we use the same amount as before)

# Create the leaf scripts A and B
# TODO: Create script A: P(remote_htlc) OP_CHECKSIG OP_1 OP_CSV OP_DROP cltv_expiry OP_CHECKLOCKTIMEVERIFY OP_DROP
# Add cltv_delta (40 blocks) to current block height
cltv_expiry = node.getblockcount() + 40


# TODO: Create script B: OP_SIZE 32 OP_EQUALVERIFY OP_HASH160 <RIPEMD160(payment_hash)> OP_EQUALVERIFY P(local_htlc) OP_CHECKSIGVERIFY P(remote_htlc) OP_CHECKSIG
# We use the same dummy payment hash as before.

# Compute TapLeaves A and B
# Method: ser_string(data) is a function which adds compactsize to input data.
hash_inputA = TAPSCRIPT_VER + ser_string(scriptA)
hash_inputB = TAPSCRIPT_VER + ser_string(scriptB)

# TODO: Use the tagged_hash function to compute TapLeaf A and B

# Method: Returns tapbranch hash. Child hashes are lexographically sorted and then concatenated.
# l: tagged hash of left child
# r: tagged hash of right child
def tapbranch_hash(l, r):
    return tagged_hash("TapBranch", b''.join(sorted([l,r])))

# TODO: Use tapbranch-hash to compute internal node TapBranch AB

# TODO: Use tagged_hash to compute TapTweak

# TODO: Tweak the alice revocation pubkey

# TODO: Create scriptPubKey (bob_accepted_htlc_spk) P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)

# TO REMOTE OUTPUT
# to remote output amount
# As we have no to_local output in this case, the to_remote value pay the fees
# Using the same fee as before
to_remote_value_sat =  channel_value_sat - anchor_output_value_satoshis * 2 - htlc_output_value_satoshis - commitment_fee
to_remote_value = to_remote_value_sat.to_bytes(8, byteorder="little", signed=False)
print("channel_value_sat, anchor_output_value_satoshis * 2, commitment_fee: ", channel_value_sat, anchor_output_value_satoshis, commitment_fee)

# P(remote) OP_CHECKSIG OP_1 OP_CSV OP_DROP
# TODO: Use CScript to create the to remote script

# Method: ser_string(data) is a function which adds compactsize to input data.
hash_input = TAPSCRIPT_VER + ser_string(script)

# To Remote Output script_root
# TODO: Use the tagged_hash function to compute the script root

# To Remote Output Tagged Hash
# TODO: Use the tagged_hash function to compute the taptweak

# TODO: Use the tweak_add to tweak the NUMS key

# TODO: Create the scriptPubKey (bob_to_remote_spk)  P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)

outputs = (
    anchor_output_value
    + varint_len(anchors_sorted[0][0])
    + anchors_sorted[0][0]
    + anchor_output_value
    + varint_len(anchors_sorted[1][0])
    + anchors_sorted[1][0]
    + htlc_output_value
    + varint_len(bob_accepted_htlc_spk)
    + bob_accepted_htlc_spk
    + to_remote_value
    + varint_len(bob_to_remote_spk)
    + bob_to_remote_spk
)

# Locktime is the same

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded, indent=2, default=str))


## The sighash for the key path spend

In [ ]:
index_of_this_input = bytes.fromhex("0000 0000")

# SIGHASH for key path spend
# See BIP-341 for details
sighash_epoch = bytes.fromhex("00")

# Control
hash_type = bytes.fromhex("00") # SIGHASH_DEFAULT (a new sighash type meaning implied SIGHASH_ALL)

# Transaction data
sha_prevouts = sha256(txid + index).digest()

sha_amounts = sha256(channel_value).digest()

# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
sha_scriptpubkeys = sha256(
    varint_len(channel_spk)
    + channel_spk
).digest()

sha_sequences = sha256(sequence).digest()

sha_outputs = sha256(outputs).digest()

# Data about this input
spend_type = bytes.fromhex("00") # no annex present

sig_msg = (
    sighash_epoch
    + hash_type
    + version
    + locktime
    + sha_prevouts
    + sha_amounts
    + sha_scriptpubkeys
    + sha_sequences
    + sha_outputs
    + spend_type
    + index_of_this_input
)

tag_hash = sha256("TapSighash".encode()).digest()
sighash = sha256(tag_hash + tag_hash + sig_msg).digest()


## Signing the sighash

In [ ]:
# Reusing the reordered participants to match the sorted pubkeys

# Alice and Bob generates its own nonce pair (secnonce, pubnonce)
secnonces, pubnonces = [], []
for pk, sk, pcs in participants:
    # TODO: Use per-commitment nonce for bob to get deterministic results and nonce_gen for Alice jit nonce

# Alice and Bob has exchanged already the pubnonces, so they can agregate them and create the session context
# TODO: Use nonce_agg to aggregate the pubnonces


# TODO: Use the SessionContext to create the gignature session

# Alice and Bob produces their partial signatures
# TODO: Use sign function to produce the partial signatures

# Each side verify the other’s partial signature before proceeding
for i, psig in enumerate(psigs):
    assert partial_sig_verify(psig, pubnonces, sorted_pubkeys, [bip86_tweak], [True], sighash, i)

# Each side combines partial signatures into the final Schnorr signature
# TODO: Use the function partial_sig_agg to aggregate the partial signatures

# Sanity check: verify with BIP340 against the *tweaked* x-only key ---
ok = schnorr_verify(sighash, agg_pubkey_tweaked, agg_sig)
print("aggregated Schnorr verifies?", ok)


## The signed transaction

In [ ]:
witness = (
    bytes.fromhex("01") # one stack item in the witness
    + varint_len(agg_sig)
    + agg_sig
)

# the final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed tx: ",signed_tx.hex())
# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(signed_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

print(node.testmempoolaccept(rawtxs=[signed_tx.hex()]))